Notebook esame: 
In questo notebook mi occuperò dell' analisi degli spettri di assorbimento di due specie di molecole (CO2 e C2H2), ottenuti attraverso il metodo 'comb-locked frequency-swept synthesis' (CLFSS), il quale permette di avere simultanaemante un grande livello di precisione e di range di misura in tempi brevi.
Come primo passo importo le librerie necessarie e i dati per un plot preliminare.

In [6]:
# Import libraries
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import ipywidgets as widgets
from ipywidgets import interact_manual
from ipywidgets import interact
from scipy.signal import find_peaks
import spettro_module as sm

In [7]:
# Dizionario dei dati
dati = {}

# Costruisco i vettori
myList=['C2H2', 'CO2'] #modifica in percorsi relativi
dati['C2H2']=np.loadtxt("ATLAS_2018/C2H2_Pband_10Torr.dat",delimiter="\t", skiprows=2, usecols=range(6))
dati['CO2']=np.loadtxt("ATLAS_2018/CO2_3THz_scan_CRDS__data_2e-2Torr.dat",delimiter="\t", skiprows=2,usecols=range(6))


In [8]:
#prova plot dati
plt.close('all')

for name in myList:
    plt.figure()
    plt.plot(dati[name][:,0],dati[name][:,-1],label=name)
    plt.legend()
    plt.grid()


Ora come prima analisi è utile trovare la posizione e l'altezza dei picchi, informazioni utili da utilizzare come starting point per il fit dei modelli.

In [9]:
plt.close('all')
#algoritmo di peaks
#per ora lasciamo da parte il calcolo della baseline
peaks={}
hmin=[.02, 6e-9]
i=0
for name in myList:
    peaks[name]=[dati[name][find_peaks(dati[name][:,-1],height=hmin[i], width=10)[0],0],dati[name][find_peaks(dati[name][:,-1],height=hmin[i], width=10)[0],-1]]
    plt.figure()
    plt.plot(dati[name][:,0],dati[name][:,-1])
    plt.plot(peaks[name][0],peaks[name][1], '.')
    i+=1

Ora si può procedere all' analisi in due metodi, che chiameremo 'fit dei picchi' e 'fit delle bande'. Il primo consiste nell'analizzare con un fit i singoli picchi di assorbimento, mentre il secondo si procede con l'analisi dei massimi appartenenti alla stessa banda. Iniziamo col primo metodo.

In [10]:
#fit dei picchi
plt.close('all')
plt.figure()
bline=[.005,0]#2.5e-9]
i=1
#for name in myList:
name='CO2'
cond=(dati[name][:,-1]<=bline[i])
dati2=dati[name][:,0]*cond
for j in peaks[name][0]:
    #perche` questa formula? voglio trovare il pt piu` vicino che sia sotto la bline (altrimenti arrivo al picco)`
    xmin=np.argmin(1/(dati2-j))
    xmax=np.argmax(1/(dati2-j)) #dati[name][cond,0])
    xfit=dati[name][xmin:xmax,0]
    yfit=dati[name][xmin:xmax,-1]

    #per il fit utilizzo la funzione di voigt, la quale in prima approssimazione puo` essere approssimata dalla funzione di pseudo voigt, una combinazione (non convoluzione) di lorentziana e gaussiana
    fit,cov=curve_fit(sm.pseudo_Voigt1,xfit,yfit) 
    #,p0=[],bounds=((0.001,-np.inf,0,0),(1,np.inf,1,1)))
    
    #plt.plot(xfit,sm.pseudo_Voigt1(xfit,*fit))
    #controllo che funzioni il metodo scelto
    #plt.plot(dati[name][:,0],1/(dati2-j))
    plt.plot(xfit,yfit)
    plt.plot(xfit,sm.pseudo_Voigt1(xfit,*fit))
#plt.plot(dati[name][:xmax,0],dati[name][:xmax,0])
i+=1    


/Users/federicotota/miniconda3/envs/scripting-course/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Illustriamo ora il secondo metodo, ovvero il fit delle bande attraverso la funzione di maxwell-boltzmann. La difficolta` qui sta nel distinguere le varie bande, trovando un metodo che possa farlo in autonomia

In [ ]:
#fit delle bande 
plt.close('all')
kb=1.380649 *10**-23
gmu=[196.8,190.29]
eps=[2e-1,1e-8]

vmax=[0,0]
norm=[0,0]
i=0

for name in myList:
    xb=np.copy(peaks[name][0])
    yb=np.copy(peaks[name][1])-bline[i]
    b_max=np.argmax(yb)
    vmax[i]=xb[b_max]-gmu[i]
    vmax[1]=np.sqrt(kb*(8.21917808*10**21))
    norm[i]=np.exp(1)*yb[b_max]*((abs(vmax[i])))/(((1/(np.pi))**(3/2))*(4*np.pi))
    #guess plot
    plt.figure()
    plt.grid()
    plt.plot(xb,yb, '.')
    plt.plot(xb,sm.max_boltz(xb,vmax[i],norm[i],gmu[i])) #(xb[b_max]-gmu[i]),norm
    bcon=(abs(yb-sm.max_boltz(xb,vmax[i],norm[i],gmu[i])))*abs(xb-xb[b_max])<eps[i] #*abs(xb-xb[b_max])
    #bcon=[for j in len(xb[bcon]):  ]
    plt.plot(xb[bcon],yb[bcon],'.r')
    
    i+=1
